In [34]:
from pytube import YouTube
from pydub import AudioSegment
import os

In [35]:
def youtube_to_wav(url, output_path):
    mp4_file = YouTube(url).streams.filter(only_audio=True).first().download(
        output_path=output_path
    )
    
    wav_file = os.path.splitext(mp4_file)[0] + ".wav"
    
    AudioSegment.from_file(mp4_file).export(wav_file, format="wav")
    
    os.remove(mp4_file)
    
    return wav_file

In [36]:
url = 'https://www.youtube.com/watch?v=pat2-f0BXCg'
output = './Output/'

wav_file = youtube_to_wav(url, output)